# Combining information from two tables 

This notebook contains code to compare and copy-paste information from one dataframe to another if the identity is the same.
This notebook contains:
<br> A) Combining information between tables of presynaptic partners
<br> B) Combining information between tables of presynaptic partners and databse of postsynaptic partners
<br> C) Combining information between tables of postsynaptic partners and databse of postsynaptic partners
<br> D) Combining information between tables for neuron authorship

In [1]:
import os
import math
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')

C:\Users\sebas\anaconda3\envs\EM\lib\site-packages\blessed\terminal.py:186: UserWarning: Failed to setupterm(kind='xterm-color'): Could not find terminal xterm-color
  warnings.warn(msg)


## A) Combining information between tables of presynaptic partners


### Chossing files of interest and data to transfer

In [3]:
# Choose path and file
dataPath = r'D:\Connectomics-Data\FlyWire\Excels\drive-data-sets'

date = '20230526'
fileName_list = [f'All_Tm9_neurons_input_count_ME_R_{date}.xlsx',
                 f'All_Tm9_neurons_input_count_ME_R_no_twig_proofread_{date}.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))


In [308]:
#Choose column information to compare and to transfer
columns_to_compare = ['presynaptic_ID', 'postsynaptic_ID', 'seg_id']
columns_to_transfer_based_on_pre = ['presynaptic_ID','postsynaptic_ID','symbol','guess','hemisphere','lab', 'author','name','twigs proofread (Y/N)', 
                                    'FlyWire proofread (Y/N)','identified_in', 'lab authorship (Y/N)', 'Extra notes as comments (initials)']
columns_to_transfer_based_on_post = ['presynaptic_ID','postsynaptic_ID', 'optic_lobe_id','column_id','detached_lamina (Y/N)', 'healthy_L3 (Y/N)']


### Loading files as distict dataframes

In [175]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["postsynaptic_ID"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

### Updating column information

In [177]:
# The distinct columns to update and the updated versions of them will be stored dictionaries

columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [179]:
#Replacing the 'INPUTS PROOFREAD' labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
    

In [180]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update
    

Updating:   0%|          | 0/1351 [00:00<?, ?it/s]

Updating:   0%|          | 0/1351 [00:00<?, ?it/s]

Updating:   0%|          | 0/1351 [00:00<?, ?it/s]

Updating:   0%|          | 0/294 [00:00<?, ?it/s]

Updating:   0%|          | 0/294 [00:00<?, ?it/s]

Updating:   0%|          | 0/294 [00:00<?, ?it/s]

### Updating dataframe information

In [182]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [184]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][column_name] =  updated_columns[f'{df_name}_{column_name}']


### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [374]:
def update_dataframe(source_df, target_df, reference_column1, reference_column2):
    # Create a dictionary mapping from the reference columns to the source DataFrame
    reference_columns = [reference_column1, reference_column2]
    reference_dict = source_df.groupby(reference_columns).first().reset_index().to_dict(orient='records')
    reference_dict = {(row[reference_column1], row[reference_column2]): row for row in reference_dict}

    # Update the target DataFrame based on the reference columns
    for i, row in target_df.iterrows():
        ref1 = row[reference_column1]
        ref2 = row[reference_column2]
        if (ref1, ref2) in reference_dict:
            source_row = reference_dict[(ref1, ref2)]
            target_df.loc[i] = source_row

    return target_df

### Provide the user-chosen columns and reference column as inputs to the function

In [375]:
source_cols = columns_to_transfer_based_on_pre 
target_cols = columns_to_transfer_based_on_pre 
reference_column1 = 'presynaptic_ID'
reference_column2 = 'postsynaptic_ID'

### Call the function with the source and target data frames and the provided inputs

In [390]:
source_df = data_frames['df_01'][source_cols].copy()
target_df = data_frames['df_02'][target_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)

result_df = update_dataframe(source_df, target_df,reference_column1, reference_column2)
presynaptic_result_df = result_df.copy()

### Repeating the same process but for postsynaptic_id-based information

In [391]:
source_cols = columns_to_transfer_based_on_post 
target_cols = columns_to_transfer_based_on_post 
reference_column1 = 'presynaptic_ID'
reference_column2 = 'postsynaptic_ID'

source_df = data_frames['df_01'][source_cols].copy()
target_df = data_frames['df_02'][target_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)

result_df = update_dataframe(source_df, target_df,reference_column1, reference_column2)
postsynaptic_result_df = result_df.copy()

### Combining the data frames and saving the data in an excel file

In [395]:
final_df = pd.concat([presynaptic_result_df,postsynaptic_result_df],axis=1)

In [398]:
#Saving in a new file

str_final_df = final_df.astype(str)

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
fileName_list[0]
file_name = f'{fileName_list[1]}_UPDATED_from_{fileName_list[0]}_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
str_final_df.to_excel(savePath, sheet_name='Data frame update')

##  B) Combining information between tables of presynaptic partners and databse of postsynaptic partners

### Chossing files of interest and data to transfer

In [39]:
# Choose path and file
dataPath = r'C:\Connectomics-Data\FlyWire\Excels\drive-data-sets'

date = '20230822'
fileName_list = [f'Tm2 proofreadings_{date}.xlsx',
                 f'Tm2_neurons_input_count_R_{date}.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))

In [40]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Loading the distinct data frames

In [41]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["seg_id"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

In [42]:
# Naming some columns different to match the two dataframes
for df_name in data_frames.keys():
    if 'postsynaptic_ID' not in data_frames[df_name].columns:
        data_frames[df_name]['postsynaptic_ID'] = data_frames[df_name]['seg_id']
        

### Updating column information

In [43]:
# The distinct columns to update and the updated versions of them will be stored dictionaries
columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [44]:
#Replacing the 'INPUTS PROOFREAD' or NaN labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
        elif id == float('nan'):
            value[i] = marker_id_updated # Replacement by the marker
        elif math.isnan(float(id)):
            value[i] = marker_id_updated # Replacement by the marker

In [45]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update

Updating:   0%|          | 0/1375 [00:00<?, ?it/s]

Updating:   0%|          | 0/15210 [00:00<?, ?it/s]

### Updating dataframe information

In [46]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [47]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][f'Updated_{column_name}'] =  updated_columns[f'{df_name}_{column_name}']

In [48]:
#The following ids have been updated
query_name = 'df_01'
diff_df = data_frames[query_name].astype(str).loc[data_frames[query_name].astype(str)['Updated_postsynaptic_ID'] != data_frames[query_name].astype(str)['postsynaptic_ID']]
print('Previous:')
print(diff_df['postsynaptic_ID'].unique())
print('Updated:')
print(diff_df['Updated_postsynaptic_ID'].unique())

Previous:
['nan' '720575940628022346']
Updated:
['INPUTS PROOFREAD' '720575940619338053']


### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [49]:
def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

In [50]:
# Function inputs
source_cols = ['optic_lobe_id', 'column_id','detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID','dorso-ventral']
target_cols = ['optic_lobe_id', 'column_id', 'detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID','dorso-ventral']
reference_column = 'postsynaptic_ID'

# Selecting dataframes and resetting index
source_df = data_frames['df_01'][source_cols].copy()
source_df.reset_index(inplace = True, drop = True)
target_df = data_frames['df_02'][target_cols].copy()
target_df.reset_index(inplace = True, drop = True)


source_df = source_df.astype(str)
target_df = target_df.astype(str)

# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df.head()

,optic_lobe_id,column_id,detached_lamina (Y/N),healthy_L3 (Y/N),postsynaptic_ID,Updated_postsynaptic_ID,dorso-ventral
0,R341,nan,N,Y,720575940647380020,720575940647380020,V
1,R341,nan,N,Y,720575940647380020,720575940647380020,V
2,R341,nan,N,Y,720575940647380020,720575940647380020,V
3,R341,nan,N,Y,720575940647380020,720575940647380020,V
4,R341,nan,N,Y,720575940647380020,720575940647380020,V


### Saving back to excel file

In [51]:
# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath_list[1])
writer = pd.ExcelWriter(filePath_list[1], engine = 'openpyxl')
writer.book = book

result_df = result_df.astype(str)
result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

## C) Combining information between tables of postsynaptic partners and databse of postsynaptic partners

In [122]:
# Choose path and file
dataPath = r'E:\Connectomics-Data\FlyWire\Excels\drive-data-sets'

date = '20230621'
fileName_list = [f'Tm9 proofreadings_{date}.xlsx',
                 f'Tm16_neurons_outputs_count_L_Tm9_{date}.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))

In [123]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Loading the distinct data frames

In [124]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["seg_id"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

In [125]:
# Naming some columns different to match the two dataframes
for df_name in data_frames.keys():
    if 'postsynaptic_ID' not in data_frames[df_name].columns:
        data_frames[df_name]['postsynaptic_ID'] = data_frames[df_name]['seg_id']
        

In [126]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Updating column information

In [127]:
# The distinct columns to update and the updated versions of them will be stored dictionaries
columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [128]:
#Replacing the 'INPUTS PROOFREAD' or NaN labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
        elif id == float('nan'):
            value[i] = marker_id_updated # Replacement by the marker
        elif math.isnan(float(id)):
            value[i] = marker_id_updated # Replacement by the marker

In [129]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update

Updating:   0%|          | 0/1375 [00:00<?, ?it/s]

Updating:   0%|          | 0/1452 [00:00<?, ?it/s]

### Updating dataframe information

In [130]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [131]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][f'Updated_{column_name}'] =  updated_columns[f'{df_name}_{column_name}']

In [132]:
#The following ids have been updated
query_name = 'df_01'
diff_df = data_frames[query_name].astype(str).loc[data_frames[query_name].astype(str)['Updated_postsynaptic_ID'] != data_frames[query_name].astype(str)['postsynaptic_ID']]
print('Previous:')
print(diff_df['postsynaptic_ID'].unique())
print('Updated:')
print(diff_df['Updated_postsynaptic_ID'].unique())

Previous:
[]
Updated:
[]


### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [133]:
def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

In [134]:
# Function inputs
source_cols = ['XYZ-ME','XYZ-LO','optic_lobe_id', 'column_id','detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID']
target_cols = ['XYZ-ME','XYZ-LO','optic_lobe_id', 'column_id', 'detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID']
reference_column = 'postsynaptic_ID'

# Selecting dataframes and resetting index
source_df = data_frames['df_01'][source_cols].copy()
source_df.reset_index(inplace = True, drop = True)
target_df = data_frames['df_02'][target_cols].copy()
target_df.reset_index(inplace = True, drop = True)


source_df = source_df.astype(str)
target_df = target_df.astype(str)

# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df.head()

,XYZ-ME,XYZ-LO,optic_lobe_id,column_id,detached_lamina (Y/N),healthy_L3 (Y/N),postsynaptic_ID,Updated_postsynaptic_ID
0,"67167.0,50125.0,4221.0","82126.0,53158.0,6017.0",L314,nan,?,nan,720575940626237898,720575940626237898
1,"66890.0,48924.0,4466.0","82040.0,52275.0,5971.0",L476,nan,?,nan,720575940615900358,720575940615900358
2,"64840.0,50020.0,4511.0","81655.0,52786.0,5912.0",L167,nan,?,nan,720575940630534518,720575940630534518
3,"65978.0,47858.0,4567.0","81724.0,51402.0,5913.0",L120,nan,N,nan,720575940632125251,720575940632125251
4,"66694.0,52382.0,4179.0","82453.0,54018.0,5985.0",L646,nan,N,nan,720575940640110670,720575940640110670


### Saving back to excel file

In [135]:

# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath_list[1])
writer = pd.ExcelWriter(filePath_list[1], engine = 'openpyxl')
writer.book = book

result_df = result_df.astype(str)
result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

### D) Combining information between tables for neuron authorship

In [5]:
# Loading the first data frame

# Choose path and file
PC_disc = 'D'
#dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\submission_nature'
dataPath = f'{PC_disc}:\FlyWire-Data\FlyWire-Authorship'
date = '20240115'
fileName = f'Segment IDs in publication_{date}.xlsx'

filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["seg_id"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
    
#Dropping dupllicates
segment_publication_df = df.drop_duplicates(subset=["seg_id"], keep='first')



In [9]:
# Loading the other dataframe

# Choose path and file
PC_disc = 'D'
#dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\submission_nature'
dataPath = f'{PC_disc}:\FlyWire-Data\FlyWire-Authorship'
date = '20240115'
fileName = f'Multi-neuron-changelog-{date}.xlsx'
filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["segment_ID"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
    
change_log_df = df.copy()
#change_log_df['segment_ID'] = change_log_df['segment_ID'].str.strip("'")

In [10]:
segment_publication_df

,symbol,hemisphere,seg_id,optic_lobe_id,backbone proofread (Y/N)
0,Tm9,R,720575940632916783,R1,Y
1,Tm9,R,720575940628906795,R2,Y
2,Tm9,R,720575940639049038,R3,Y
3,Tm9,R,720575940641658676,R4,Y
4,Tm9,R,720575940638132058,R5,Y
...,...,...,...,...,...
5183,TmY17,R,720575940633505934,TmY17-R152,Y
5184,TmY17,R,720575940651270902,TmY17-R153,Y
5185,TmY17,R,720575940625696637,TmY17-R154,Y
5186,TmY17,R,720575940625313428,TmY17-R155,Y


In [11]:
# Adding tte ID column from the fisrt table to the second

# Merge the data frames based on the common column "Updated_seg_ids"
change_log_final = change_log_df.copy()
merged_df = pd.merge(change_log_final, segment_publication_df, left_on='segment_ID', right_on='seg_id', how='left')

# Add a new column "neuron type" to change_log_df using information from the "symbol" column
change_log_final['neuron type'] = merged_df['symbol']

In [12]:
change_log_final

,segment_ID,total_edits,user_name,user_id,number_of_edits,percent_of_total,user_name.1,user_id.1,number_of_edits.1,percent_of_total.1,...,percent_of_total.9,user_name.10,user_id.10,number_of_edits.10,percent_of_total.10,user_name.11,user_id.11,number_of_edits.11,percent_of_total.11,neuron type
0,720575940603557920,1,Dustin Garner,100.0,1.0,100.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tm9
1,720575940622667784,3,J. Anthony Ocho,1323.0,2.0,66.67,Anjali Pandey,1153.0,1.0,33.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tm9
2,720575940627031570,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tm9
3,720575940627067919,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tm9
4,720575940627086863,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tm9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,720575940620166592,8,Kendrick Joules Vinson,1932.0,5.0,62.50,Gerit Linneweber,129.0,3.0,37.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TmY17
5181,720575940623665151,3,Michelle Pantujan,1325.0,1.0,33.33,remer tancontian,1928.0,1.0,33.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TmY17
5182,720575940623772612,2,Philip,2765.0,2.0,100.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TmY17
5183,720575940632894419,25,Shaina Mae Monungolh,1926.0,12.0,48.00,Rashmita Rana,1417.0,7.0,28.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TmY17


In [13]:
#Saving in a new file

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'change_log_final_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
change_log_final.to_excel(savePath, sheet_name='User-Lab summary')